# NLP HW2: Parsing
Last Updated: Sep 25 3:20 PM

In [2]:
!pip install jdc
import jdc

zsh:1: command not found: pip


In [3]:
!wget -nc https://raw.githubusercontent.com/aarsri/nlp_hw2/refs/heads/main/utils.py
!wget -nc https://raw.githubusercontent.com/aarsri/nlp_hw2/refs/heads/main/trees.py

zsh:1: command not found: wget
zsh:1: command not found: wget


## Part 1: Bi-LSTM POS Tagger
In this part, you will build a part-of-speech tagger using the bidirectional LSTM architecture.

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import time
import random
import pickle
import nltk
from utils import Vocab, read_pos_file
from nltk.corpus import treebank
"""You should not need any other imports."""

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

nltk.download('treebank')
brown = list(treebank.tagged_sents())


[nltk_data] Downloading package treebank to /Users/johan/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


**[4 points]** Implement a **bidirectional** LSTM with embedding size 128 and hidden size 256 using torch.nn.LSTM with appropriate parameters set. Remember that PyTorch's cross entropy loss includes softmax.

In [5]:
class BiLSTMTagger(nn.Module):
    def __init__(self, words, tags, embedding_dim, hidden_dim):
        super().__init__()
        self.words=words
        self.tags=tags
        self.words.add('<UNK>') 
        self.tags.add('<UNK>')


        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
    

        # Layers
        self.embedding = nn.Embedding(len(self.words), embedding_dim)
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=True
        )
        self.dropout = nn.Dropout(0.35)
        self.w_out = nn.Linear(2 * hidden_dim, len(self.tags))



In [6]:
%%add_to BiLSTMTagger
def forward(self, sentence):
	embeddings = self.embedding(torch.tensor(sentence, dtype=torch.long))
	lstm_in = embeddings.view(len(sentence), 1, -1)
	lstm_out, _ = self.lstm(lstm_in)
	lstm_out = lstm_out.view(len(sentence), -1)
	lstm_out = self.dropout(lstm_out)
	scores = self.w_out(lstm_out)
	return scores
 

In [7]:
%%add_to BiLSTMTagger
def predict(self, scores):
    return torch.argmax(scores, dim=1)

**[3 points]** Write **training** and **evaluation** procedures. These should be similar to HW1 Part 2 and 3.
- For debugging purposes only, shorten the training data by commenting out the train_sents += brown line. Accuracy will be high because the train set is otherwise a combination of ATIS and BROWN, while the test set is ATIS. Commenting out this line will still have the model trained and tested on ATIS, yielding a high accuracy.  However, it will not be generalized well enough to do the free response.


In [8]:
%%add_to BiLSTMTagger
def fit(self, data, lr=0.01, epochs=5):
    optimizer = torch.optim.Adam(self.parameters(), lr=lr)
    loss_function = nn.CrossEntropyLoss()

    for epoch in range(1, epochs + 1):
        self.train()
        random.shuffle(data)

        total_loss = 0.0
        total_tokens = 0
        start_time = time.time()

        for sentence in data:
            words = [word.lower() for word, _ in sentence]
            tags = [tag for _, tag in sentence]
            word_idxs = [self.words.numberize(w) for w in words]
            inputs = torch.tensor(word_idxs, dtype=torch.long).to(next(self.parameters()).device)
            targets = torch.tensor([self.tags.numberize(tag) for tag in tags], dtype=torch.long).to(next(self.parameters()).device)
            self.zero_grad()
            scores = self(inputs)   
            loss = loss_function(scores, targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm=5.0)
            optimizer.step()
            total_loss += loss.item() * len(targets)
            total_tokens += len(targets)

        average_loss = total_loss / total_tokens
        print(f"Epoch {epoch}/{epochs}\tAverage Loss: {average_loss:.4f}\tTime: {time.time() - start_time:.2f}s")


In [9]:
%%add_to BiLSTMTagger
def evaluate(self, data):
    self.eval()
    total_correct = 0
    total_tokens = 0

    with torch.no_grad():
        for sentence in data:
            words, tags = zip(*sentence)
            word_idxs = [self.words.numberize(w.lower()) for w in words]
            sentence = torch.tensor(word_idxs, dtype=torch.long).to(next(self.parameters()).device)
            targets = torch.tensor([self.tags.numberize(tag) for tag in tags], dtype=torch.long).to(device)
            scores = self(sentence)
            predicted = self.predict(scores)
            total_correct += sum(predicted == targets).item()
            total_tokens += len(targets)
    return total_correct / total_tokens

**[1 point]** Report **loss per epoch** and **accuracy**. For full credit, accuracy must be at least 92% on val.pos and test.pos. Include your **saved model** in the submission files.
- Because the dataset is small, scores from randomly initialized runs can have greater variance. If you're close to this score (e.g., 90%), there is likely not a bug and instead you had an unlucky run. You can run it again if you have time or comment that you believe the code is correct.


In [28]:

def main():
    train_data = read_pos_file("data/train.pos")
    val_data = read_pos_file("data/val.pos")
    test_data = read_pos_file("data/test.pos")
    
    words=Vocab()
    tags=Vocab()
        
    for sentence in train_data:
        for word, tag in sentence:
            words.add(word)
            tags.add(tag)
            
    model = BiLSTMTagger(words, tags, embedding_dim=128, hidden_dim=256).to(device)
    model.fit(train_data, lr=0.01, epochs=5)
    torch.save({
	 	'model_state_dict': model.state_dict(),
	 	'words': model.words,
	 	'tags': model.tags,
	 	'embedding_dim': 128,
	 	'hidden_dim': 256
	 }, 'bilstm_model.pth')
    
    val_acc = model.evaluate(val_data)
    test_acc = model.evaluate(test_data)
    print(f"Validation Accuracy: {val_acc:.4f}")
    print(f"Test Accuracy: {test_acc:.4f}")
    
    for i, sentence in enumerate(test_data[:10]):
        words = [w.lower() for w, _ in sentence]
        word_idxs = [model.words.numberize(w) for w in words]
        inputs = torch.tensor(word_idxs, dtype=torch.long).to(next(model.parameters()).device)
        true_tags = [t for _, t in sentence]  # get actual tags
        predicted_tags = model.predict(model(inputs))
        predicted_tag_strs = [model.tags.denumberize(tag) for tag in predicted_tags]

        print(f"\nSentence {i+1}:")
        for word, true_tag, predicted_tag in zip(words, true_tags, predicted_tag_strs):
            print(f"{word} {true_tag} {predicted_tag}")

main()
     

<string>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch 1/5	Average Loss: 0.6251	Time: 2.16s
Epoch 2/5	Average Loss: 0.2728	Time: 2.36s
Epoch 3/5	Average Loss: 0.2348	Time: 2.11s
Epoch 4/5	Average Loss: 0.2054	Time: 2.32s
Epoch 5/5	Average Loss: 0.2069	Time: 2.04s
Validation Accuracy: 0.9346
Test Accuracy: 0.9375

Sentence 1:
the DT DT
flight NN NN
should MD MD
arrive VB VB
at IN IN
eleven CD CD
a.m RB RB
tomorrow NN NN
. PUNC PUNC

Sentence 2:
i PRP PRP
would MD MD
like VB VB
to TO TO
find VB VB
a DT NNP
flight NN NN
that WDT WDT
goes VBZ VBZ
from IN IN
la NNP NNP
guardia NNP NNP
airport NN NNP
to TO TO
san NNP NNP
jose NNP NNP
. PUNC PUNC

Sentence 3:
show VB VB
me PRP PRP
the DT DT
flights NNS NNS
from IN IN
newark NNP NNP
to TO TO
los NNP NNP
angeles NNP NNP
. PUNC PUNC

Sentence 4:
what WDT WP
airline NN NN
is VBZ VBZ
this DT DT
? PUNC PUNC

Sentence 5:
show VB VB
me PRP PRP
the DT DT
t NNP NNP
w NNP NNP
a NNP NNP
flight NN NN
. PUNC PUNC

Sentence 6:
i PRP PRP
would MD MD
like VB VB
to TO TO
travel VB VB
to TO TO
westchester NNP

**[2 points]** Free response: For the first 10 sentences of test.pos, **report the POS tags predicted by your model.**

Sentence 1:
the DT DT
flight NN NN
should MD MD
arrive VB VB
at IN IN
eleven CD CD
a.m RB RB
tomorrow NN NN
. PUNC PUNC

Sentence 2:
i PRP PRP
would MD MD
like VB VB
to TO TO
find VB VB
a DT NNP
flight NN NN
that WDT WDT
goes VBZ VBZ
from IN IN
la NNP NNP
guardia NNP NNP
airport NN NNP
to TO TO
san NNP NNP
jose NNP NNP
. PUNC PUNC

Sentence 3:
show VB VB
me PRP PRP
the DT DT
flights NNS NNS
from IN IN
newark NNP NNP
to TO TO
los NNP NNP
angeles NNP NNP
. PUNC PUNC

Sentence 4:
what WDT WP
airline NN NN
is VBZ VBZ
this DT DT
? PUNC PUNC

Sentence 5:
show VB VB
me PRP PRP
the DT DT
t NNP NNP
w NNP NNP
a NNP NNP
flight NN NN
. PUNC PUNC

Sentence 6:
i PRP PRP
would MD MD
like VB VB
to TO TO
travel VB VB
to TO TO
westchester NNP NNP
. PUNC PUNC

Sentence 7:
list VB VB
american NNP NNP
airlines NNP NNP
flights NNS NNS
from IN IN
new NNP NNP
york NNP NNP
newark NNP NNP
to TO TO
nashville NNP NNP
. PUNC PUNC

Sentence 8:
thanks UH UH
. PUNC PUNC

Sentence 9:
what WDT WP
flights NNS NNS
are VBP VBP
there EX EX
from IN IN
nashville NNP NNP
to TO TO
houston NNP NNP
tomorrow NN NN
evening NN NN
that WDT WDT
serve VBP VBP
dinner NN NN
? PUNC PUNC

Sentence 10:
i PRP PRP
'd MD MD
like VB VB
to TO TO
fly VB VB
next JJ JJ
friday NNP NNP
. PUNC PUNC

- What works well, and what doesn't?
> Words that have definitive purposes like punctuation, proper nouns, and verbs almost always get the right tags. The incorrectly tagged words are the ones where there's more ambiguity like nouns and proper nouns
- For words tagged incorrectly, why do you think it happens, and what tag do they tend to get?
> This can happen when words can have different parts of speech based on context. Airport for example can be a common noun (the airport) or a proper noun (JFK Airport), so it can be easy for the parser to get them mixed up
- Think about micro- and macro-level tags (e.g., is it tagging NNS as NN, or VBD as NN, and which one is worse?).
> It's okay when there are micro level tag issues because the general meaning of the sentence won't be skewed too much, but when you have macro level discrepancies, meanings can become misunderstood


## Part 2: Probabilistic Context Free Grammar
In this part, you will learn a PCFG given training data annotated with parse trees.

In [11]:
import trees
import fileinput
import collections
from collections import defaultdict, Counter
import re
"""You should not need any other imports, but you may import anything that helps."""

counts = collections.defaultdict(collections.Counter)

**[4 points]** Write code to read in trees and to count all the rules used in each tree. **Terminal nodes should be POS tags rather than words** to allow us to use the POS tagger. This means the grammar must contain a set of rules mapping each nonterminal POS tags in the PTB tag set to a terminal POS tag:
```
DT -> DT_t
NN -> NN_t
NNS -> NNS_t
```


In [12]:
def count_rules():

    counts=defaultdict(Counter)  
    probs = defaultdict(dict)
    cfg = defaultdict(list)  
    
    with open("data/train.trees", "r") as f:
        for line in f:
            tree = trees.Tree.from_str(line.strip())
            if tree.root is None:
                continue
            
            for node in tree.bottomup():
                if len(node.children)>0:
                    if len(node.children[0].children)==0:
                        label=node.label
                        if "_" in label:
                            label = label.split("_")[-1]
                        label+="_t"
                        rhs = (label, )
                    else:
                        rhs=tuple(child.label for child in node.children)
                        
                    counts[node.label][rhs]+=1
                    
    for lhs in counts:
        total = sum(counts[lhs].values())
        for rhs in counts[lhs]:
            probs[lhs][rhs] = counts[lhs][rhs]/total
            cfg[rhs].append(lhs)
                    
           
    return counts, probs, cfg         
    


**[3 points]** Write code to compute the conditional probability of each rule and **print the PCFG in a readable format**, such as:
```
NP -> DT NN # 0.5
NP -> DT NNS # 0.5
```

In [13]:
def print_cfg(counts, probs, cfg):
    for rhs, parents in cfg.items():
        for A in parents:
            prob = probs[A].get(rhs)
            count = counts[A].get(rhs)
            if len(rhs) == 1:
                print(f"  {A} -> {rhs[0]} "
                      f"[count={count}, prob={prob:.4f}]")
            elif len(rhs) == 2:
                B, C = rhs
                print(f"  {A} -> ({B}, {C}) "
                      f"[count={count}, prob={prob:.4f}]")
            else:
                print(f"  {A} -> {' '.join(rhs)} "
                      f"[count={count}, prob={prob:.4f}]")
                
    print()


**[3 points]** Running the code on train.trees, **report**:
- How many unique rules are there?
> 419
- What are the top five most frequent rules, and how many times did each occur?
> 1.	IN --> IN_t	(Occurence: 482 times)
  2.	PUNC --> PUNC_t	(Occurence: 469 times)
  3.	NP_NNP --> NNP_t(Occurence: 451 times)
  4.	NNP --> NNP_t	(Occurence: 408 times)
  5.	NN --> NN_t	(Occurence: 281 times)
- What are the top five highest-probability rules with left-hand side NP, and what are their probabilities?
> 1.	NP --> NNP NNP	(Probability: 0.1928)
  2.	NP --> NP NP*	(Probability: 0.1159)
  3.	NP --> DT NN	(Probability: 0.1014)
  4.	NP --> DT NNS	(Probability: 0.0855)
  5.	NP --> DT NP*	(Probability: 0.0841)
- **Free Response**: Did the most frequent rules surprise you? Why or why not?
> Not really, the most frequent rules would be things like Punctuation, Proper nouns, and other nouns are the most common parts of speech in sentences



In [22]:
def main():
  counts, probs, cfg = count_rules()
  
  num=0
  for lhs in probs:
    for rhs in probs[lhs]:
      print(f"{lhs} -> {" ".join(rhs)} # {probs[lhs][rhs]}")
      num+=1
      
  print(f"Unique rules: {num}")
  
  print("CFG: ")
  print_cfg(counts, probs, cfg)
  
  rules = []
  for lhs in counts:
    for rhs in counts[lhs]:
      count = counts[lhs][rhs]
      rule = f"{lhs} --> {' '.join(rhs)}"
      rules.append((rule, count))
      
  rules = sorted(rules, key = lambda x:x[1], reverse=True)
  
  print(f"Number of rules: {len(rules)}")
  print("Top five most frequent rules: ")
  for r, (rule, count) in enumerate(rules[:5], 1):
    print(f"{r}.\t{rule}\t(Occurence: {count} times)")
  print()
  
  np_rules = list(sorted([(rule, prob) for rule, prob in probs['NP'].items()], key = lambda x:x[1], reverse=True))
  print("Top five most frequent left-side NP rules:")
  for r, (rule, prob) in enumerate(np_rules[:5], 1):
    print(f"{r}.\tNP --> {" ".join(rule)}\t(Probability: {prob:.4f})")
  

    
  

main()
      



VB -> VB_t # 1.0
DT -> DT_t # 1.0
NNS -> NNS_t # 1.0
NP -> DT NNS # 0.08550724637681159
NP -> NP NP* # 0.11594202898550725
NP -> DT NN # 0.10144927536231885
NP -> NP PP # 0.04057971014492753
NP -> CD NNS # 0.002898550724637681
NP -> NNP NNP # 0.1927536231884058
NP -> NP_NNP NP # 0.013043478260869565
NP -> NP SBAR # 0.01884057971014493
NP -> NP_CD PP # 0.0014492753623188406
NP -> DT JJ # 0.007246376811594203
NP -> NNP NP* # 0.0463768115942029
NP -> QP RB # 0.004347826086956522
NP -> JJ NNP # 0.004347826086956522
NP -> DT NP* # 0.08405797101449275
NP -> CD RB # 0.030434782608695653
NP -> PDT NP* # 0.004347826086956522
NP -> NP VP # 0.005797101449275362
NP -> CD NP* # 0.02318840579710145
NP -> DT NX # 0.0014492753623188406
NP -> NN NN # 0.013043478260869565
NP -> NP_NNP NP_NNP # 0.007246376811594203
NP -> NN NP* # 0.017391304347826087
NP -> NNP JJ # 0.002898550724637681
NP -> NNP POS # 0.002898550724637681
NP -> NN NNS # 0.010144927536231883
NP -> NP_NN NP* # 0.007246376811594203
NP -> NP

# Part 3: CKY Parsing
In this part, you will implement the CKY parsing algorithm.

**[4 points]** Implement a CKY parser that takes your grammar and a set of POS tags as input, and outputs the highest-probability parse. If you can't find any parse, output a blank line. Use **log-probabilities** to avoid underflow.

In [15]:
def parse_cky(sentences):
    counts, probs, cfg = count_rules()

    def reconstruct(label, i, k, words):
        bp = backptr[(i, k)][label]
        if isinstance(bp, str): 
            return f"({label} {bp})"
        else:
            (left_label, j, right_label) = bp
            left_subtree  = reconstruct(left_label,  i, j, words)
            right_subtree = reconstruct(right_label, j, k, words)
            return f"({label} {left_subtree} {right_subtree})"

    for index, sentence in enumerate(sentences):
        chart = dict()
        backptr = dict()

        words, tags = zip(*sentence)
        n = len(sentence)

        for i, tag in enumerate(tags):
            preterm = (tag + "_t",)  
            chart[(i, i+1)] = {}
            backptr[(i, i+1)] = {}
            for lhs in cfg.get(preterm, []):  
                chart[(i, i+1)][lhs] = math.log(probs[lhs][preterm])
                backptr[(i, i+1)][lhs] = words[i]

        for l in range(2, n+1):          
            for i in range(n-l+1):      
                k = i + l          
                chart[(i, k)] = {}
                backptr[(i, k)] = {}
                for j in range(i+1, k): 
                    for left_label in chart[(i, j)]:
                        for right_label in chart[(j, k)]:
                            if (left_label, right_label) in cfg:
                                for A in cfg[(left_label, right_label)]:
                                    candidate_score = (chart[(i, j)][left_label] + chart[(j, k)][right_label] + math.log(probs[A][(left_label, right_label)]))
                                    if A not in chart[(i, k)] or candidate_score > chart[(i, k)][A]:
                                        chart[(i, k)][A] = candidate_score
                                        backptr[(i, k)][A] = (left_label, j, right_label)

        print(f"Sentence {index+1}:")
        if "TOP" in chart[(0, n)]:
            tree_str = reconstruct("TOP", 0, n, words)
            print(f"{tree_str}\tProbability: {chart[(0, n)]["TOP"]}\n") 
        else:
            print()


**[1 point]** Test the CKY parser with the test set with **gold POS tags**. **Print** the parse trees for the **first 10 test sentences** (use empty line for no parse). For full credit, these should match or be similar to the true parses in test.trees.

In [24]:
def test_parser():
  print("first 10 test sentences with gold POS tags")
  print()
  sentences = read_pos_file("data/test.pos")[0:10]
  parse_cky(sentences)
  

**[2 points]** Integrate your **Bi-LSTM tagger** to assign POS tags to the words of any input sentence before passing it in to your CKY parser.

In [25]:
def assign_tags():
  checkpoint = torch.load('bilstm_model.pth', map_location=device, weights_only=False)
  words = checkpoint['words']
  tags = checkpoint['tags']
  embedding_dim = checkpoint['embedding_dim']
  hidden_dim = checkpoint['hidden_dim']
  model = BiLSTMTagger(words, tags, embedding_dim, hidden_dim).to(device)
  model.load_state_dict(checkpoint['model_state_dict'])
  return model

**[1 point]** Test the full pipeline (**input → tagger → CKY**). **Print** the parse trees for the **first 10 test sentences** (use empty line for no parse) using the predicted POS tags.

In [23]:
def test_pipeline():

  test_parser()
  
  model = assign_tags()
  print()
  print("first 10 test sentences with predicted POS tags")
  print()
  
  sentences = read_pos_file("data/test.pos")[:10]
  sentences_with_predicted_tags = []
  for sentence in sentences:
    words, tags = zip(*sentence)
    word_idxs = [model.words.numberize(word.lower()) for word in words]
    sentence = torch.tensor(word_idxs, dtype=torch.long).to(device)
    scores = model(sentence)
    predicted = model.predict(scores)
    predicted_tags = [model.tags.denumberize(tag) for tag in predicted]
    sentences_with_predicted_tags.append(list(zip(words, predicted_tags)))

  parse_cky(sentences_with_predicted_tags)

test_pipeline()

first 10 test sentences with gold POS tags

Sentence 1:
(TOP (S (NP (DT the) (NN flight)) (VP (MD should) (VP (VB arrive) (VP* (PP (IN at) (NP (CD eleven) (RB a.m))) (NP_NN tomorrow))))) (PUNC .))	Probability: -20.432453165513543

Sentence 2:
(TOP (S (NP_PRP i) (VP (MD would) (VP (VB like) (S_VP (TO to) (VP (VB find) (NP (NP (DT a) (NN flight)) (SBAR (WHNP_WDT that) (S_VP (VBZ goes) (VP* (PP (IN from) (NP (NNP la) (NP* (NNP guardia) (NN airport)))) (PP (TO to) (NP (NNP san) (NNP jose)))))))))))) (PUNC .))	Probability: -37.07221194904133

Sentence 3:
(TOP (S_VP (VB show) (VP* (NP_PRP me) (NP (NP (DT the) (NNS flights)) (NP* (PP (IN from) (NP_NNP newark)) (PP (TO to) (NP (NNP los) (NNP angeles))))))) (PUNC .))	Probability: -14.05035596365033

Sentence 4:

Sentence 5:
(TOP (S_VP (VB show) (VP* (NP_PRP me) (NP (NP (DT the) (NP* (NNP t) (NNP w))) (NP* (NNP a) (NN flight))))) (PUNC .))	Probability: -15.293025795448274

Sentence 6:
(TOP (S (NP_PRP i) (VP (MD would) (VP (VB like) (S_VP (TO to)

<string>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).


**[2 points]** Free response: For the first 10 sentences of test.pos…
- Is there a difference in which sentences it fails to parse given gold tags vs. your tagger's outputs? Why or why not?
> No, both of them were able to parse the same sentences. This is because the grammar rules are broad enough that both POS assignments can still fall under the cfg rules
- Which ones does it do well on (i.e., match the true parse in test.trees), and why?
> it did well on sentences that were short and had words with specific meanings like prepositions and proper nouns. This is because shorter words mean less context is needed to understand the sentence, and specific words reduce ambiguity, which makes it easier for the parser to capture the meaning/structure of the sentence
- Which ones does it do poorly on (but still produces a parse), and why?
> I'm noticing that the longer the sentence. is, there is usually less confidence, which I think is just do to added sentence structure complexity


**Congratulations! That's a wrap on HW2. Onto neural methods and greater generalizability.**